In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from shapely.geometry import Polygon

In [2]:
pwd

u'/Users/kristikorsberg/devel/big_data/big_data_prj/Analyzing_Noise_in_NYC/py'

In [3]:
ct = gpd.read_file('../data/nyct2010_17a/nyct2010.shp')

In [4]:
ct.head(2)

,BoroCT2010,BoroCode,BoroName,CDEligibil,CT2010,CTLabel,NTACode,NTAName,PUMA,Shape_Area,Shape_Leng,geometry
0,5000900,5,Staten Island,I,000900,9,SI22,West New Brighton-New Brighton-St. George,3903,2.497010e+06,7729.016794,"POLYGON ((962269.1260375977 173705.5001831055,..."
1,1009800,1,Manhattan,I,009800,98,MN19,Turtle Bay-East Midtown,3808,1.906016e+06,5534.199811,"POLYGON ((994133.507019043 214848.8975830078, ..."


In [5]:
noise = pd.read_csv('/Users/kristikorsberg/Downloads/311_Service_Requests_from_2010_to_Present.csv')

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (8,17,40,43,44,45,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
noise.head(2)

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,35607069,03/02/2017 03:33:17 PM,03/12/2017 11:21:04 AM,HPD,Department of Housing Preservation and Develop...,FLOORING/STAIRS,FLOOR,RESIDENTIAL BUILDING,11220,6123 4 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.640087,-74.019194,"(40.6400872732656, -74.01919432059279)"
1,35597174,03/01/2017 05:10:34 PM,03/12/2017 09:40:02 AM,HPD,Department of Housing Preservation and Develop...,DOOR/WINDOW,DOOR,RESIDENTIAL BUILDING,10467,3301 PERRY AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.877016,-73.875373,"(40.877016358193174, -73.87537267430177)"


In [7]:
noise.columns

Index([u'Unique Key', u'Created Date', u'Closed Date', u'Agency',
       u'Agency Name', u'Complaint Type', u'Descriptor', u'Location Type',
       u'Incident Zip', u'Incident Address', u'Street Name', u'Cross Street 1',
       u'Cross Street 2', u'Intersection Street 1', u'Intersection Street 2',
       u'Address Type', u'City', u'Landmark', u'Facility Type', u'Status',
       u'Due Date', u'Resolution Description',
       u'Resolution Action Updated Date', u'Community Board', u'Borough',
       u'X Coordinate (State Plane)', u'Y Coordinate (State Plane)',
       u'Park Facility Name', u'Park Borough', u'School Name',
       u'School Number', u'School Region', u'School Code',
       u'School Phone Number', u'School Address', u'School City',
       u'School State', u'School Zip', u'School Not Found',
       u'School or Citywide Complaint', u'Vehicle Type',
       u'Taxi Company Borough', u'Taxi Pick Up Location',
       u'Bridge Highway Name', u'Bridge Highway Direction', u'Road Ramp',

In [8]:
noise.head(2)

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,35607069,03/02/2017 03:33:17 PM,03/12/2017 11:21:04 AM,HPD,Department of Housing Preservation and Develop...,FLOORING/STAIRS,FLOOR,RESIDENTIAL BUILDING,11220,6123 4 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.640087,-74.019194,"(40.6400872732656, -74.01919432059279)"
1,35597174,03/01/2017 05:10:34 PM,03/12/2017 09:40:02 AM,HPD,Department of Housing Preservation and Develop...,DOOR/WINDOW,DOOR,RESIDENTIAL BUILDING,10467,3301 PERRY AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.877016,-73.875373,"(40.877016358193174, -73.87537267430177)"


In [9]:
noise = noise[['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Complaint Type', 'Descriptor', 'Incident Zip', \
               'X Coordinate (State Plane)', 'Y Coordinate (State Plane)']]

In [10]:
noise = noise.dropna()

In [11]:
geometry = [Point(xy) for xy in zip(noise['X Coordinate (State Plane)'], noise['Y Coordinate (State Plane)'])]
noise = noise.drop(['X Coordinate (State Plane)', 'Y Coordinate (State Plane)'], axis=1)
crs = {'init': 'epsg:2263'}
geo_noise = gpd.GeoDataFrame(noise, crs=crs, geometry=geometry)

In [12]:
print geo_noise.crs
print ct.crs

{'init': 'epsg:2263'}
{u'lon_0': -74, u'datum': u'NAD83', u'y_0': 0, u'no_defs': True, u'proj': u'lcc', u'x_0': 300000, u'units': u'us-ft', u'lat_2': 41.03333333333333, u'lat_1': 40.66666666666666, u'lat_0': 40.16666666666666}


In [13]:
ct = ct.to_crs({'init':'epsg:2263'})

In [14]:
print geo_noise.crs
print ct.crs

{'init': 'epsg:2263'}
{'init': 'epsg:2263'}


In [15]:
ct = ct[['geometry', 'CT2010', 'Shape_Area', 'Shape_Leng']]

In [16]:
ct.columns

Index([u'geometry', u'CT2010', u'Shape_Area', u'Shape_Leng'], dtype='object')

In [17]:
noise_simple = geo_noise[['Unique Key', 'geometry']]

In [18]:
#http://geoffboeing.com/2016/10/r-tree-spatial-index-python/
precise_matches = {}
spatial_index = geo_noise.sindex
for i, CT2010 in enumerate(ct.CT2010):
    possible_matches_index = list(spatial_index.intersection(ct['geometry'][i].bounds))
    possible_matches = geo_noise.iloc[possible_matches_index]
    precise_matches[CT2010] = possible_matches[possible_matches.intersects(ct['geometry'][i])]
    #precise_matches[CT2010]['CT2010'] = CT2010 

In [19]:
noise_ct = pd.concat(precise_matches).reset_index()

In [20]:
noise_ct.head(2)

,level_0,level_1,Agency,Closed Date,Complaint Type,Created Date,Descriptor,Incident Zip,Unique Key,geometry
0,000100,203592,DOB,04/10/2017 02:02:26 PM,Special Enforcement,04/10/2017 02:02:26 PM,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280)
1,000100,178552,DEP,03/29/2017 05:00:00 PM,Lead,03/28/2017 05:00:00 PM,Lead Kit Request (Residential) (L10),11201,35812477.0,POINT (986751 194303)


In [21]:
noise_ct = noise_ct.rename(columns={'level_0':'CT2010'})
noise_ct = noise_ct.drop('level_1',axis=1)

In [22]:
noise_ct.head(2)

,CT2010,Agency,Closed Date,Complaint Type,Created Date,Descriptor,Incident Zip,Unique Key,geometry
0,000100,DOB,04/10/2017 02:02:26 PM,Special Enforcement,04/10/2017 02:02:26 PM,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280)
1,000100,DEP,03/29/2017 05:00:00 PM,Lead,03/28/2017 05:00:00 PM,Lead Kit Request (Residential) (L10),11201,35812477.0,POINT (986751 194303)


In [23]:
noise_ct['CT2010'] = noise_ct['CT2010'].astype(str)

In [24]:
noise_ct['Census Tract'] = noise_ct['CT2010'].apply(lambda s: s.lstrip("0"))

In [25]:
noise_ct.head(2)

,CT2010,Agency,Closed Date,Complaint Type,Created Date,Descriptor,Incident Zip,Unique Key,geometry,Census Tract
0,000100,DOB,04/10/2017 02:02:26 PM,Special Enforcement,04/10/2017 02:02:26 PM,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280),100
1,000100,DEP,03/29/2017 05:00:00 PM,Lead,03/28/2017 05:00:00 PM,Lead Kit Request (Residential) (L10),11201,35812477.0,POINT (986751 194303),100


In [26]:
pop = pd.read_csv('../data/New_York_City_Population_By_Census_Tracts.csv')

In [27]:
pop.head(2)

,Borough,Year,FIPS County Code,DCP Borough Code,Census Tract,Population
0,Bronx,2000,5,2,100,12780
1,Bronx,2000,5,2,200,3545


In [28]:
pop = pop[pop['Year']==2010]

In [29]:
pop['Census Tract'] = pop['Census Tract'].astype(str)
noise_ct['Census Tract'] = noise_ct['Census Tract'].astype(str)

In [30]:
pop.head(2)

,Borough,Year,FIPS County Code,DCP Borough Code,Census Tract,Population
2168,Bronx,2010,5,2,100,11091
2169,Bronx,2010,5,2,200,4334


In [31]:
ct_noise_pop = pd.merge(noise_ct, pop, on='Census Tract', how='outer')

In [32]:
ct_noise_pop.head(2)

,CT2010,Agency,Closed Date,Complaint Type,Created Date,Descriptor,Incident Zip,Unique Key,geometry,Census Tract,Borough,Year,FIPS County Code,DCP Borough Code,Population
0,000100,DOB,04/10/2017 02:02:26 PM,Special Enforcement,04/10/2017 02:02:26 PM,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280),100,Bronx,2010,5,2,11091
1,000100,DOB,04/10/2017 02:02:26 PM,Special Enforcement,04/10/2017 02:02:26 PM,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280),100,Brooklyn,2010,47,3,4338


In [33]:
type(ct_noise_pop)

geopandas.geodataframe.GeoDataFrame

In [34]:
print len(ct_noise_pop['Unique Key'].unique())
print len(noise['Unique Key'].unique())
print len(noise_ct['Unique Key'].unique())

135487
224925
135486


In [35]:
bbl = pd.read_csv('../data/ct_bbl_avg.csv')
bbl = bbl.drop('Unnamed: 0',axis=1)
bbl.head()

,CT2010,BldgAge,NumFloors,AssessTot,BldgArea,BldgDepth,BldgFront,ComArea,FactryArea,GarageArea,NumBldgs,OfficeArea,OtherArea,StrgeArea
0,100,126.363229,4.078475,1.045777e+06,23211.573991,50.908117,36.010314,10799.793722,0.000000,5640.847534,1.233184,3255.802691,1474.094170,64.596413
1,200,65.637972,2.006498,2.328835e+04,2007.178066,42.968656,21.679009,114.395047,27.079009,14.140330,1.176887,1.650943,42.903302,4.317217
2,201,104.861111,5.444444,2.597126e+06,44266.583333,79.905833,61.648333,19089.111111,0.000000,0.000000,1.166667,0.000000,18583.333333,0.000000
3,202,88.125000,5.671875,5.540089e+06,96952.791667,77.809375,105.229375,31991.687500,0.000000,205.416667,1.833333,701.770833,29854.645833,31.250000
4,300,82.781022,2.874088,1.183985e+06,26030.167883,65.316350,57.577372,17628.919708,441.481752,2193.781022,1.189781,7453.321168,5114.620438,1588.145985


In [36]:
ct_noise_pop.head(2)

,CT2010,Agency,Closed Date,Complaint Type,Created Date,Descriptor,Incident Zip,Unique Key,geometry,Census Tract,Borough,Year,FIPS County Code,DCP Borough Code,Population
0,000100,DOB,04/10/2017 02:02:26 PM,Special Enforcement,04/10/2017 02:02:26 PM,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280),100,Bronx,2010,5,2,11091
1,000100,DOB,04/10/2017 02:02:26 PM,Special Enforcement,04/10/2017 02:02:26 PM,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280),100,Brooklyn,2010,47,3,4338


In [37]:
ct_noise_pop.CT2010 = ct_noise_pop.CT2010.astype(str)
dropped = []
for elem in ct_noise_pop.CT2010:
    dropped.append(elem.lstrip('00'))

In [38]:
dropped[:10]

['100', '100', '100', '100', '100', '100', '100', '100', '100', '100']

In [39]:
ct_noise_pop.CT2010 = dropped

In [40]:
ct_noise_pop.head(2)

,CT2010,Agency,Closed Date,Complaint Type,Created Date,Descriptor,Incident Zip,Unique Key,geometry,Census Tract,Borough,Year,FIPS County Code,DCP Borough Code,Population
0,100,DOB,04/10/2017 02:02:26 PM,Special Enforcement,04/10/2017 02:02:26 PM,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280),100,Bronx,2010,5,2,11091
1,100,DOB,04/10/2017 02:02:26 PM,Special Enforcement,04/10/2017 02:02:26 PM,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280),100,Brooklyn,2010,47,3,4338


In [41]:
bbl.CT2010 = bbl.CT2010.astype(str)
merged = pd.merge(bbl, ct_noise_pop, on='CT2010')

In [42]:
print len(merged)
print len(bbl)
print len(ct_noise_pop)

232197
1329
232746


In [43]:
merged.head()

,CT2010,BldgAge,NumFloors,AssessTot,BldgArea,BldgDepth,BldgFront,ComArea,FactryArea,GarageArea,...,Descriptor,Incident Zip,Unique Key,geometry,Census Tract,Borough,Year,FIPS County Code,DCP Borough Code,Population
0,100,126.363229,4.078475,1.045777e+06,23211.573991,50.908117,36.010314,10799.793722,0.0,5640.847534,...,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280),100,Bronx,2010,5,2,11091
1,100,126.363229,4.078475,1.045777e+06,23211.573991,50.908117,36.010314,10799.793722,0.0,5640.847534,...,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280),100,Brooklyn,2010,47,3,4338
2,100,126.363229,4.078475,1.045777e+06,23211.573991,50.908117,36.010314,10799.793722,0.0,5640.847534,...,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280),100,Manhattan,2010,61,1,2
3,100,126.363229,4.078475,1.045777e+06,23211.573991,50.908117,36.010314,10799.793722,0.0,5640.847534,...,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280),100,Queens,2010,81,4,4949
4,100,126.363229,4.078475,1.045777e+06,23211.573991,50.908117,36.010314,10799.793722,0.0,5640.847534,...,Lead Kit Request (Residential) (L10),11201,35812477.0,POINT (986751 194303),100,Bronx,2010,5,2,11091


In [44]:
merged.columns

Index([u'CT2010', u'BldgAge', u'NumFloors', u'AssessTot', u'BldgArea',
       u'BldgDepth', u'BldgFront', u'ComArea', u'FactryArea', u'GarageArea',
       u'NumBldgs', u'OfficeArea', u'OtherArea', u'StrgeArea', u'Agency',
       u'Closed Date', u'Complaint Type', u'Created Date', u'Descriptor',
       u'Incident Zip', u'Unique Key', u'geometry', u'Census Tract',
       u'Borough', u'Year', u'FIPS County Code', u'DCP Borough Code',
       u'Population'],
      dtype='object')

In [45]:
merged = merged.drop(['Census Tract', 'Borough', 'Year', 'FIPS County Code', 'DCP Borough Code',\
                     'geometry'], axis=1)

In [48]:
merged.to_csv('../data/bbl_ct_noise.csv')

In [47]:
#ct_noise_pop.to_file('../data/ct_noise_pop/ct_noise_pop.shp',driver='ESRI Shapefile')